This file shows how to extract grid data and velocity data from the raw MITgcm output files. You need MITgcm downloaded (specifically, MITgcmutils)

In [1]:
import MITgcmutils
import numpy as np
import numpy.ma as ma

from scipy.io import loadmat
import glob

In [ ]:
# get time data
datt = MITgcmutils.mnc.mnc_files('../data/raw_data/outs_sn.*.nc')
time = datt.variables['T'][:]
datt.close()

In [ ]:
# get basin geometry data from Matlab file
basin_data = loadmat('../MITgcm_code/setup/basin_400km_400km.mat')
x_basin0 = int(basin_data['x_basin0']) # grid cell of the left extent of the basin 
x_basin1 = int(basin_data['x_basin1']) # grid cell of the right extent of the basin
y_inlet = int(basin_data['y_inlet']) # grid cell of the north extent of the basin

In [ ]:
# get grid information
gridm = MITgcmutils.mnc.mnc_files('../data/raw_data/grid*')
hFacc = gridm.variables['HFacC'][:] # partial cell fill (cell-centered)
hFacw = gridm.variables['HFacW'][:] # partial cell fill (west edge)
hFacs = gridm.variables['HFacS'][:] # partial cell fill (south edge)
z = gridm.variables['Z'][:] # array of z values
depth = gridm.variables['Depth'][:] # 2D array of depth
drf =gridm.variables['drF'][:] # vertical spacing
x = gridm.variables['X'][:] # x grid cells
y = gridm.variables['Y'][:] # y grid cells
gridm.close()

In [ ]:
# save grid and time information
np.savez('grid_info.npz', time=time, x_basin0=x_basin0, x_basin1=x_basin1, y_inlet=y_inlet,
        hFacc=hFacc, drf=drf, hFacw=hFacw, hFacs=hFacs, z=z, depth=depth, x=x, y=y)

In [ ]:
# load and save velocity and temperature data for the entire domain at the last time step
datt = MITgcmutils.mnc.mnc_files('../data/raw_data/outs_sn.*.nc')
uvel = datt.variables['UVEL'][-1,:,:,:]
vvel = datt.variables['VVEL'][-1,:,:,:]
wvel = datt.variables['WVEL'][-1,:,:,:]
theta = datt.variables['THETA'][-1,:,:,:]

np.savez('last_time_step.npz', uvel=uvel, vvel=vvel, wvel=wvel, theta=theta)

In [ ]:
# load and save velocity and temperature data for the basin over all time steps

datt = MITgcmutils.mnc.mnc_files('../data/raw_data/outs_sn.*.nc')
# each field indices are [time, z, y, x]
uvel_basin = datt.variables['UVEL'][:,:,0:y_inlet, x_basin0:x_basin1]
vvel_basin = datt.variables['VVEL'][:,:,0:y_inlet, x_basin0:x_basin1]
wvel_basin = datt.variables['WVEL'][:,:,0:y_inlet, x_basin0:x_basin1]
theta_basin = datt.variables['THETA'][:,:,0:y_inlet, x_basin0:x_basin1]
datt.close()

np.savez('basin_data.npz', uvel_basin=uvel_basin, vvel_basin=vvel_basin, wvel_basin=wvel_basin,theta_basin=theta_basin)